In [ ]:
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec
import os
# API Keys
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")

# Cấu hình Google Gemini API
genai.configure(api_key=GOOGLE_API_KEY)

# Khởi tạo Pinecone client mới
pc = Pinecone(api_key=PINECONE_API_KEY)

# Kiểm tra xem index đã tồn tại chưa
index_name = "chatbotfinance"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Kích thước vector của "all-MiniLM-L6-v2" là 384
        metric="cosine"
    )

# Kết nối đến index
index = pc.Index(index_name)

# Kiểm tra thông tin index
print("✅ Pinecone Index đã sẵn sàng!")
print(index.describe_index_stats())



✅ Pinecone Index đã sẵn sàng!
{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


In [22]:
import pandas as pd

csv_path = "2022-Vietnam1.csv"
df = pd.read_csv(csv_path, encoding="utf-8-sig")

# ✅ XỬ LÝ DỮ LIỆU: Xóa cột trống, đặt lại tiêu đề
column_headers = df.iloc[0].tolist()  # Lấy tiêu đề từ hàng đầu tiên
df_cleaned = df.iloc[1:].reset_index(drop=True)  # Xóa hàng đầu tiên khỏi dữ liệu chính
print(df_cleaned.head())


  Unnamed: 0 Unnamed: 1         Unnamed: 2 Unnamed: 3 Unnamed: 4  \
0          1        ACB                ACB       HOSE  Ngân hàng   
1          2        BCM        Becamex IDC       HOSE  Tài chính   
2          3        BID               BIDV       HOSE  Ngân hàng   
3          4        BVH  Tập đoàn Bảo Việt       HOSE  Tài chính   
4          5        CTG         VietinBank       HOSE  Ngân hàng   

     Unnamed: 5         Unnamed: 6         Unnamed: 7 CÂN ĐỐI KẾ TOÁN  \
0     Ngân hàng          Ngân hàng          Ngân hàng               5   
1  Bất động sản       Bất động sản       Bất động sản               5   
2     Ngân hàng          Ngân hàng          Ngân hàng               5   
3      Bảo hiểm  Bảo hiểm nhân thọ  Bảo hiểm nhân thọ               5   
4     Ngân hàng          Ngân hàng          Ngân hàng               5   

  Unnamed: 9  ...    Unnamed: 158          Unnamed: 159        Unnamed: 160  \
0       2022  ...             NaN                   NaN                 N

In [23]:
import torch
from sentence_transformers import SentenceTransformer
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Đang chạy trên: {device.upper()}")

# ✅ CẤU HÌNH BERT (Chạy trên GPU nếu có)
bert_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)

✅ Đang chạy trên: CUDA


In [24]:
def get_bert_embedding(text, model, device="cuda"):
    embedding = model.encode(text, convert_to_tensor=True, device=device)
    return embedding.tolist()


In [25]:
from tqdm import tqdm
import time
# ✅ CHIA NHỎ DỮ LIỆU & TẠO EMBEDDINGS
batch_size = 1  # Số dòng gửi mỗi lần
total_rows = len(df_cleaned)
output_file = "embeddings_data.txt"

with open(output_file, "w", encoding="utf-8") as f:
    for i in tqdm(range(0, total_rows, batch_size)):
        batch_vectors = []
        batch = df_cleaned.iloc[i:i+batch_size]

        for idx, row in batch.iterrows():
            # 🚀 Gộp dữ liệu đúng format: "Tên cột: Giá trị"
            text = " | ".join([f"{col_name}: {row.iloc[col_index]}" 
                               for col_index, col_name in enumerate(column_headers)])
            text = text.replace("#####", "MISSING_DATA")

            try:
                # 🚀 Tạo embedding từ BERT
                embedding = get_bert_embedding(text, bert_model, device)

                # 🚀 Lưu vào danh sách với metadata
                batch_vectors.append((str(idx), embedding, {"data": text}))

                # ✅ Ghi vào file txt
                f.write(f"ID: {idx} | Text: {text} | Embedding: {embedding}\n")

            except Exception as e:
                print(f"⚠️ Lỗi xử lý dòng {idx}: {str(e)}")
                continue

        # ✅ Gửi batch vào Pinecone
        if batch_vectors:
            try:
                index.upsert(vectors=batch_vectors)
            except Exception as e:
                print(f"⚠️ Lỗi khi upsert vào Pinecone: {str(e)}")
                continue

        time.sleep(1)

print(f"✅ Đã thêm {total_rows} vector embeddings vào Pinecone thành công!")
print("✅ Dữ liệu đã được lưu vào:", output_file)
print(index.describe_index_stats())

 34%|███▎      | 565/1675 [21:08<41:32,  2.25s/it]   


KeyboardInterrupt: 

In [ ]:
index.upsert(vectors)
print(f"✅ Đã thêm {len(vectors)} vector embeddings vào Pinecone thành công!")
print(index.describe_index_stats())